<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/pivot-point-driven-pendulum-Mattmflores/blob/main/pivot_point_driven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Helper Functions

In [60]:
# Function: get_pivot_xy
# Parameters:
#     t(array.    )    : time values
#     omega_d(float)   : driving angular frequency
#     x_p0, y_p0(float): optional offsets for pivot’s origin
#     l(float)         : pendulum length
# Returns:
#     x_p, y_p(ndarrays): pivot coordinates over time
# Description:
#     Computes the motion of the pendulum pivot point
def get_pivot_xy(t, omega_d, x_p0=0.0, y_p0=0.0, l=1.0):
    x_p = -l * np.sin(omega_d * t) + x_p0
    y_p =  l * (1 - np.cos(omega_d * t)) + y_p0
    return x_p, y_p


In [61]:
# Function: get_pendulum_xy
# Parameters:
#     t(array)         : time array
#     theta(array).    : angular displacement
#     l(float)         : pendulum length
# Returns:
#     x_rel, y_rel(ndarrays): pendulum bob coordinates relative to the pivot
# Description:
#     Computes the bob’s position relative to the pivot
def get_pendulum_xy(t, theta, l=1.0):
    x_rel = l * np.sin(theta)
    y_rel = -l * np.cos(theta)
    return x_rel, y_rel

In [62]:
# Function: get_lab_xy
# Parameters:
#     x_pivot, y_pivot(array): pivot coordinates
#     x_rel, y_rel(array)    : bob coordinates relative to pivot
# Returns:
#     x_lab, y_lab(ndarrays): bob coordinates in the lab frame
# Description:
#     Converts pendulum coordinates from the pivot frame to the
#     laboratory frame
def get_lab_xy(x_pivot, y_pivot, x_rel, y_rel):
    x_lab = x_pivot + x_rel
    y_lab = y_pivot + y_rel
    return x_lab, y_lab

## Pendulum Equation of Motion

In [63]:
# Function: pendulum_rhs
# Parameters:
#     t(float)      : current time
#     y(array)      : [theta, theta_dot]
#     l(float)      : pendulum length
#     g(float).     : acceleration due to gravity
#     omega_d(float): drive angular frequency
# Returns:
#     dydt(list): time derivatives [theta_dot, theta_ddot]
# Description:
#     Computes the derivatives of the pivot-driven pendulum system.
def pendulum_rhs(t, y, l, g, omega_d):
    theta, theta_dot = y  # unpack the state vector

    # second derivative (angular acceleration)
    theta_ddot = - (g / l) * np.sin(theta) - (omega_d ** 2) * np.sin(omega_d * t + theta)

    # return time derivatives as list
    return [theta_dot, theta_ddot]


## Integration of the Pendulum Equation

---



---



In [64]:
# Simulation parameters
g = 9.81                  # gravitational acceleration (m/s^2)
l = 1.0                   # pendulum length (m)
omega_d = 3.0             # drive angular frequency (rad/s)
t_min, t_max = 0.0, 10.0  # simulation time range (s)
x_p0 = 0.0
y_p0 = 0.0

# Initial conditions
y0 = [0.0, 0.0]

In [65]:
# Integration
sol = solve_ivp(
    fun=lambda t, y: pendulum_rhs(t, y, l, g, omega_d),
    t_span=(t_min, t_max),
    y0=y0,
    dense_output=True,
    max_step=0.01
)

In [66]:
# Evaluate solution
t_plot = np.linspace(t_min, t_max, 1000)
theta_plot = sol.sol(t_plot)[0]  # θ(t)

## Animation

In [67]:
# create pendulum animation
def create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=1, show_traj=False, speed_factor=1.0):
    """
    Creates an animation of the pivot-driven pendulum.

    PARAMETERS:
    t : ndarray
        Time array.
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    x_lab : ndarray
        x-positions of the pendulum bob in the lab frame over time.
    y_lab : ndarray
        y-positions of the pendulum bob in the lab frame over time.
    l : float, optional
        pendulum length (default is 1).
    show_traj : bool, optional
        toggle showing the trajectory of the pendulum bob (default is False).
    speed_factor : float, optional
        factor to scale the animation speed (default is 1.0).
        a value > 1.0 slows down the animation, < 1.0 speeds it up.

    RETURNS:
    anim : FuncAnimation
        matplotlib animation object.
    """
    fig, ax = plt.subplots(figsize=(4, 4)) # Increased figure size
    ax.set_xlim(np.min(x_pivot) - l, np.max(x_pivot) + l)
    ax.set_ylim(np.min(y_pivot) - l, np.max(y_pivot) + l)
    # ax.set_aspect('equal', adjustable='box')
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    # Initialize plot elements
    pivot, = ax.plot([], [], 'o', color='black', markersize=8, label='Pivot')
    pendulum_arm, = ax.plot([], [], '-', color='black', lw=2, label='Pendulum Arm')
    pendulum_bob, = ax.plot([], [], 'o', color='red', markersize=12, label='Pendulum Bob')
    trajectory = None # Initialize trajectory to None

    if show_traj:
        trajectory, = ax.plot([], [], '-', color='gray', lw=1, alpha=0.5, label='Trajectory') # Add trajectory line

    def animate(i):
        # Update the positions of the plot elements
        pivot.set_data([x_pivot[i]], [y_pivot[i]]) # Pass as sequences
        pendulum_arm.set_data([x_pivot[i], x_lab[i]], [y_pivot[i], y_lab[i]])
        pendulum_bob.set_data([x_lab[i]], [y_lab[i]]) # Pass as sequences

        artists = [pivot, pendulum_arm, pendulum_bob] # List of artists to update

        if show_traj and trajectory:
             trajectory.set_data(x_lab[:i+1], y_lab[:i+1]) # Update trajectory data
             artists.append(trajectory) # Add trajectory to the list of artists

        return artists # Return all updated artists

    # Create the animation
    # Adjust the interval based on the average time step in t
    # This aims to make the animation speed consistent with the simulation time
    average_time_step = np.mean(np.diff(t))
    # Scale the interval by the speed_factor
    interval = average_time_step * 1000 * speed_factor # Convert to milliseconds and apply speed_factor.

    anim = FuncAnimation(fig, animate, frames=len(t), interval=interval, blit=True)
    plt.close(fig) # Close the initial figure to prevent it from displaying

    return anim

In [68]:
# animate motion
# plot pendulum angle versus time
# get coordinates for plotting
n_points = 300 # number of frames
t_plot = np.linspace(t_min, t_max, n_points)
theta_plot = sol.sol(t_plot)[0] # requires `dense_output=True` in `solve_ivp`
x_pivot, y_pivot = get_pivot_xy(t_plot, omega_d, x_p0, y_p0) # pivot coordinates
x_pendulum, y_pendulum = get_pendulum_xy(t_plot, theta_plot, l) # pendulum coordinates of pendulum (referenced to pivot point)
x_lab, y_lab = get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum) # pendulum coordinates in the lab frame

# animation flags/parameters
show_traj = True # True --> show trajectory; False --> do not show trajectory
speed_factor = 5 # >1 --> slow down animation; <1 --> speed up animation

# create_pendulum_animation
pendulum_animation = create_pendulum_animation(t_plot, x_pivot, y_pivot, x_lab, y_lab, show_traj=show_traj, speed_factor=speed_factor) # create animation object with trajectory shown
HTML(pendulum_animation.to_html5_video()) # display animation



## Analytic Stability Analysis

At $t = 0$, the pivot is at $(0, 0)$ and the bob is at $(0, -l)$, hanging straight down. This is the equilibrium position of the pendulum in the lab frame.

If the pendulum starts from rest $(\theta_0 = 0, \dot{\theta}_0 = 0)$, the bob will not remain stationary because the moving pivot continuously accelerates, producing a periodic forcing on the pendulum.

- For **very slow** ($\omega_d \ll \sqrt{g/l}$) or **very fast** ($\omega_d \gg \sqrt{g/l}$) drive, the pendulum stays **stable**.
- For drive **frequencies near the natural frequency** $\omega_0 = \sqrt{g/l}$ or **near twice** the natural frequency ($\omega_d \approx 2\omega_0$), the motion can become **unstable**.
- Increasing the drive amplitude makes the equilibrium less stable.

